# 다나와 사이트 자동차 실적 크롤링
## - carvarzo

In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import openpyxl

In [10]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

my_url = 'http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2018-01-00&MonthTo=' #2018년 1월 모델별 판매실적 사이트
resp = requests.get(my_url,headers=headers)
html = BeautifulSoup(resp.content,'html.parser')


In [11]:
#1월 목록만 가져오기

grid = html.find('div',{'class':'recordSection'})


In [12]:
#순위
ranks = grid.select('td.rank')

ranking = []
for rank in ranks:
    ranking.append(rank.getText())



In [13]:
#자동차 모델명
modelName = []
names = grid.findAll('input')
for name in names:
     modelName.append(name.get('title'))



In [14]:
numbers = grid.findAll('td',{'class':'right'})


산으로 가는듯한 나의 크롤링..   < td class = "right" > 의 text가 불러와지지 않아서 삽질을 했다,,

In [15]:
#판매량 크롤링

salesNum = []
rate = []    #점유율
lm = []    #전월 판매량
lm_rate = []   #전월 대비 상승/하락
ly = []    #전년도 판매량
ly_rate = []    #전년도 대비 상승/하락

for i in range(len(numbers)):
    if i%3==0:
        rate.append(numbers[i].getText())
    elif i%3==1:
        k=numbers[i].find('span').getText().replace(",","")
        lm_rate.append(k)
        lm.append(numbers[i].getText().replace(",","")[:-len(k)])
        
    else:
        y=numbers[i].find('span').getText().replace(",","")
        ly_rate.append(y)
        ly.append(numbers[i].getText().replace(",","")[:-len(y)])

#### 굳이굳이 메소드로 만들어보기..

In [16]:
def danawaRecord(url):
    resp = requests.get(url,headers=headers)
    html = BeautifulSoup(resp.content,'html.parser')
    
    grid = html.find('div',{'class':'recordSection'}) #record 섹션만 가져오기
    
    #랭킹
    ranks = grid.select('td.rank')
    ranking = []
    for rank in ranks:
        ranking.append(rank.getText())
      
    #모델명
    modelName = []
    names = grid.findAll('input')
    for name in names:
        modelName.append(name.get('title'))
    
    #판매실적
    numbers = grid.findAll('td',{'class':'right'})
    salesNum = []
    rate = []    #점유율
    lm = []    #전월 판매량
    lm_rate = []   #전월 대비 상승/하락
    ly = []    #전년도 판매량
    ly_rate = []    #전년도 대비 상승/하락

    for i in range(len(numbers)):
        if i%3==0:
            rate.append(numbers[i].getText())
        elif i%3==1:
            k=numbers[i].find('span').getText().replace(",","")
            lm_rate.append(k)
            lm.append(numbers[i].getText().replace(",","")[:-len(k)])
        else:
            y=numbers[i].find('span').getText().replace(",","")
            ly_rate.append(y)
            ly.append(numbers[i].getText().replace(",","")[:-len(y)])
            
    myung = {'순위':ranking, "모델명":modelName, "점유율":rate, "전월 판매량":lm, "전월 대비 상승,하락":lm_rate, "전년도 판매량": ly, "전년도 대비 상승,하락": ly_rate}
    record = pd.DataFrame(myung)
    
    #excel 파일 저장
    record.to_excel(writer, sheet_name = str(a)+str(l), header=True, encoding='utf-8', index=False)
    writer.save()

전체 년/월 데이터 가져오기

In [86]:
final_url = "http://auto.danawa.com/auto/?Work=record&Tab=Model&Month="

path = 'C:/Users/이명아/Desktop/카바조/danawa2.xlsx'
writer = pd.ExcelWriter(path, engine = 'openpyxl')

# i 는 년도, j는 월을 의미. 2019년도 자료를 크롤링 하고 싶으면 i 의 범위를 2020까지 해주면 된다
for a in range(2017,2019):  
    for l in range(1,13):
        if l<10:
            geturl = final_url + str(a) + "-0" + str(l) + "-00&MonthTo="
        else :
            geturl = final_url + str(a) + "-" + str(l) + "-00&MonthTo="
        danawaRecord(geturl)
        print(geturl) 
        

http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2017-01-00&MonthTo=
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2017-02-00&MonthTo=
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2017-03-00&MonthTo=
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2017-04-00&MonthTo=
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2017-05-00&MonthTo=
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2017-06-00&MonthTo=
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2017-07-00&MonthTo=
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2017-08-00&MonthTo=
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2017-09-00&MonthTo=
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2017-10-00&MonthTo=
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2017-11-00&MonthTo=
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2017-12-00&MonthTo=
http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2018-01-00&MonthTo=